In [ ]:
#Generic Libraries
import numpy as np
import pandas as pd 
import os

In [ ]:
#pip install nselib

In [ ]:
#NSE Libraries
import nselib
from nselib import capital_market
from nselib import derivatives
from datetime import datetime

#Gmail related libraries
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
dateToRun=datetime.today().strftime("%d-%m-%Y")
df=derivatives.fno_bhav_copy(dateToRun)
df.head()

Filtering Data

In [ ]:
#Keep rows where instrument = FUTSTK and Expiry = current month current year
df_fno=df[df.INSTRUMENT=='FUTSTK']

#Filtering on expiry date as current month and year
currmonthyear=str(datetime.today().strftime('%b')) +"-"+ str(datetime.today().year)
df_currmonthexp=df_fno[df_fno['EXPIRY_DT'].str.contains(currmonthyear)]

#bar% = (High - Low)/Low *100
#OI% = (CHG_IN_OI/OPEN_INT)*100
df_currmonthexp['bar%']=((df_currmonthexp['HIGH']-df_currmonthexp['LOW'])/df_currmonthexp['LOW'])*100
df_currmonthexp['OI%']=(df_currmonthexp['CHG_IN_OI']/df_currmonthexp['OPEN_INT'])*100

#bar% > 2 AND OI% outside -5 and 5
df_currmonthexp=df_currmonthexp.loc[(df_currmonthexp['bar%'] >=2) & ((df_currmonthexp['OI%'] <= -5) | (df_currmonthexp['OI%'] >=5))]
df_currmonthexp.head()

#Sorting results on OI%
df_currmonthexp=df_currmonthexp.sort_values('OI%')

#Keeping required columns only
dfFinal=df_currmonthexp[['SYMBOL','EXPIRY_DT','OPEN','HIGH','LOW','CLOSE','bar%','OI%']]
dfFinal.head()

In [ ]:
#Exporting data into excel
filename= str(datetime.now().date())
print(filename)
dfFinal.to_excel(r"EXCEL FILE PATH", index=False)

In [ ]:
#Sending Gmail
sender_email = "SENDER EMAIL ID"
sender_password = "SENDER EMAIL APP PASSWORD"
recipient_email = "aRECIPIENT EMAIL ID"
subject = "FNO Bhav Copy - "+dateToRun
body = "FNO Bhav copy attached"


with open(r"EXCEL FILE PATH", "rb") as attachment:
    # Add the attachment to the message
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header(
    "Content-Disposition",
    f"attachment; filename= "+filename+".xlsx",
)

message = MIMEMultipart()
message['Subject'] = subject
message['From'] = sender_email
message['To'] = recipient_email
html_part = MIMEText(body)
message.attach(html_part)
message.attach(part)

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
   server.login(sender_email, sender_password)
   server.sendmail(sender_email, recipient_email, message.as_string())